<a href="https://colab.research.google.com/github/sadrabr/LLM_projects/blob/main/Create_brochure_with_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv

In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00


In [3]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
import gradio as gr

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [6]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

The 'ollama serve' process is running in the background.


In [7]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff:   2% ▕▏  36 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   4% ▕▏  87 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   6% ▕▏ 124 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   8% ▕▏ 162 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   9% ▕▏ 179 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  10% ▕▏ 199 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  12% ▕▏ 236 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  14% ▕▏ 272 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  14% ▕▏ 292 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  16% ▕▏ 330 MB/2.0 GB                  p

In [8]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [15]:
system_message_normal = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [16]:
system_message_humorous = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [12]:
def stream_normal(prompt):
    messages = [
        {"role": "system", "content": system_message_normal},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='llama3.2',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [13]:
def stream_humorous(prompt):
    messages = [
        {"role": "system", "content": system_message_humorous},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='llama3.2',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [14]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [17]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="Normal":
        result = stream_normal(prompt)
    elif model=="Humorous":
        result = stream_humorous(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [21]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["Normal", "Humorous"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://526253282ef03174b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
